<a href="https://colab.research.google.com/github/dukes101/LOM-Analytics/blob/main/MatchupDataPull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Pulling


In [1]:
## Dependencies
import requests
import pandas as pd

In [2]:
## Mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
## Define years of league
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023]

## Match id to player name
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Name': ['Luca Hurst', 'Dylan Peters', 'Cody Morton', 'Matt Willadsen', 'Nate Stone', 'Hayden Bingham', 'Gavin Tacto', 'Zachary Toma', 'Alexander Nicoll', 'Evan Williams']
}

## Create data frame of id and player name
dfIdNameMatch = pd.DataFrame(data)

## Define the column names needed
matchup_column_names = {
  'matchupPeriodId':'Week',
  'away.teamId':'Team1',
  'away.totalPoints':'Score1',
  'home.teamId':'Team2',
  'home.totalPoints':'Score2',
  'playoffTierType': 'playoffType'
  }

## Define an empty data frame to append our matchup data to
data = {
    'Week': [],
    'Team1': [],
    'Score1': [],
    'Team2': [],
    'Score2': [],
    'playoffType': [],
    'Type': [],
    'Year': []
}

dfAllMatchups = pd.DataFrame(data)

In [4]:
## Define our cookies
swid = "{A12B9964-EFD3-467B-90C1-1BA157A35B47}"
espn_s2 = "AECI498yf3h0oXPHqBitIYL2FIAG2asDzwR9YFChmjfzq5G8xwbqEAz5%2FBoCTXQVj1fPy3gShVl5Uzf%2B7FRAcodgXgIJikfHu2TJlPG5YX1fWhh9htOb4hZ3vDiX5eLxjyDlYQoRmL3xyLWOywQwEu5IISq7cbyQJNBnXncQUd7xhdOqtAoiKbUACMrqUrX8RghUoTB%2Fi7JiV3OsG27PALMbKND6ujI%2BMJSPCAHrOjB4Q1svNAXbkzhPkInp497aSv2pmQBrhFBN9UMiz%2ByGTdl%2BG6qCTszXBh0mshg%2Bk6FYJw%3D%3D"

cookies = {
    "swid": swid,
    "espn_s2": espn_s2
}

In [5]:
## Loop through all years
for year in years:

  ## 2017 requires a different API call
  if year == 2017:

    url = "https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + str(193193) + "?seasonId=" + str(year)
    matchup_response = requests.get(url, params={"view": "mMatchup"}) #matchup data
    matchup_json = matchup_response.json()
    dfMatchup = pd.json_normalize(matchup_json[0]['schedule'])
    dfMatchup['playoffTierType'] = 'NONE'

  ## All other years
  else:

    # Define the URL with our parameters
    url = f"https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/segments/0/leagues/{193193}"

    # Pull team and matchup data from the URL
    matchup_response = requests.get(url
                                  ,params={"view": "mMatchupScore"} #matchup data
                                  ,cookies=cookies) #permissions

    # Transform the response into a json
    matchup_json = matchup_response.json()

    # Transform both of the json outputs into DataFrames
    dfMatchup = pd.json_normalize(matchup_json['schedule'])

  # Reindex based on column names defined above
  dfMatchup = dfMatchup.reindex(columns=matchup_column_names).rename(columns=matchup_column_names)

  # Add a new column for regular/playoff game based on week number
  # In future, determine playoffs by playoffTierType
  if year == 2023:
    dfMatchup['Type'] = ['Regular' if week<=11 else 'Playoff' for week in dfMatchup['Week']]

  elif year in [2018, 2019]:
    dfMatchup['Type'] = ['Regular' if week<=12 else 'Playoff' for week in dfMatchup['Week']]

  elif year in [2017, 2020]:
    dfMatchup['Type'] = ['Regular' if week<=13 else 'Playoff' for week in dfMatchup['Week']]

  elif year == 2021:
    dfMatchup['Type'] = ['Regular' if week<=14 else 'Playoff' for week in dfMatchup['Week']]

  else: #2022
    dfMatchup['Type'] = ['Regular' if week<=15 else 'Playoff' for week in dfMatchup['Week']]

  ## Creates a year column for the given year
  dfMatchup['Year'] = year
  dfMatchup['Year'] = dfMatchup['Year'].astype(str)

  ## Concatenates all years together into a single dataframe
  dfAllMatchups = pd.concat([dfAllMatchups, dfMatchup], ignore_index=True)

In [6]:
# Manually add playoffs for 2017 data
dfAllMatchups.iloc[52, 5] = 'WINNERS_BRACKET'
dfAllMatchups.iloc[53, 5] = 'WINNERS_BRACKET'
dfAllMatchups.iloc[54, 5] = 'LOSERS_CONSOLATION_LADDER'
dfAllMatchups.iloc[55, 5] = 'LOSERS_CONSOLATION_LADDER'
dfAllMatchups.iloc[56, 5] = 'WINNERS_BRACKET'
dfAllMatchups.iloc[57, 5] = 'WINNERS_CONSOLATION_LADDER'
dfAllMatchups.iloc[58, 5] = 'LOSERS_CONSOLATION_LADDER'
dfAllMatchups.iloc[59, 5] = 'LOSERS_CONSOLATION_LADDER'

# Data Cleaning

In [7]:
## Drop NAs
dfAllMatchups = dfAllMatchups.dropna(subset=['Team1'])

In [8]:
## Merge Team1 IDs with Names
dfAllMatchupsMerge1 = dfAllMatchups.merge(dfIdNameMatch, left_on='Team1', right_on='id', suffixes=('', '_Team1'))

## Merge Team2 IDs with Names
dfAllMatchupsMerge2 = dfAllMatchupsMerge1.merge(dfIdNameMatch, left_on='Team2', right_on='id', suffixes=('', '_Team2'))

## Drop ID columns
dfAllMatchupsDrop = dfAllMatchupsMerge2.drop(['id', 'id_Team2', 'Team1', 'Team2'], axis=1)
dfAllMatchupsRename = dfAllMatchupsDrop.rename(columns={'Name': 'Team1', 'Name_Team2': 'Team2'})

## Reorder cols
new_order = ['Week', 'Team1', 'Score1', 'Team2', 'Score2', 'Type', 'playoffType', 'Year']
dfAllMatchupsNewOrder = dfAllMatchupsRename[new_order]

## Drop consolation playoff matchups
dfAllMatchupsFinal = dfAllMatchupsNewOrder[~((dfAllMatchupsNewOrder['Type'] == 'Playoff') & (dfAllMatchupsNewOrder['playoffType'].str.lower().str.contains('consolation')))].reset_index(drop=True)

In [9]:
## Checks duplicate team names: Want to see 'True'
print((dfAllMatchups['Team1'] != dfAllMatchups['Team2']).all())

True


In [15]:
dfAllMatchupsFinal.head()

,Week,Team1,Score1,Team2,Score2,Type,playoffType,Year
0,1.0,Luca Hurst,111.0,Matt Willadsen,96.0,Regular,NONE,2017
1,11.0,Luca Hurst,95.0,Matt Willadsen,113.0,Regular,NONE,2019
2,11.0,Luca Hurst,136.1,Matt Willadsen,128.3,Regular,NONE,2020
3,10.0,Dylan Peters,115.0,Matt Willadsen,125.0,Regular,NONE,2017
4,8.0,Dylan Peters,111.0,Matt Willadsen,155.0,Regular,NONE,2018


In [11]:
dfAllMatchupsFinal.to_csv('/content/drive/My Drive/dfAllMatchups.csv', index=False)